In [1]:
sc

In [2]:
rawEventsRdd = sc.textFile("/home/axinggu/thesis/code/test/filteredEvents.txt")
rawEventsRdd.take(5)

[u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tcom.google.android.googlequicksearchbox\t1421521691\t47.235050\t-122.534698\tTacoma\t5\t11\tApp_Opened',
 u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tcom.android.phone\t1421558502\t47.235050\t-122.534698\tTacoma\t5\t21\tApp_Opened',
 u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tcom.android.systemui\t1421017853\t47.237461\t-122.530899\tTacoma\t6\t15\tApp_Opened',
 u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tcom.android.settings\t1421555801\t47.235050\t-122.534698\tTacoma\t5\t20\tApp_Opened',
 u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tcom.android.dialer\t1421555768\t47.235050\t-122.534698\tTacoma\t5\t20\tApp_Opened']

In [3]:
from collections import namedtuple

EventDataRow = namedtuple("EventDataRow", ["userId", "itemId", "ts", "latitude", "longitude", "city", "day_of_week", "time_of_day" , "event_type"])

def parseRawData(line):
    lineSplit = line.split("\t")
    return EventDataRow(userId=lineSplit[0],
                      itemId=lineSplit[1],
                      ts=int(lineSplit[2]),
                      latitude=float(lineSplit[3]),
                      longitude=float(lineSplit[4]),
                      city=lineSplit[5],
                      day_of_week=int(lineSplit[6]),
                      time_of_day=int(lineSplit[7]),
                      event_type=lineSplit[-1],
    )
    

eventsRdd = rawEventsRdd.map(parseRawData).cache()
eventsRdd.take(3)

[EventDataRow(userId=u'001e6d8e-cbe7-4374-8c38-f37962a457e9', itemId=u'com.google.android.googlequicksearchbox', ts=1421521691, latitude=47.23505, longitude=-122.534698, city=u'Tacoma', day_of_week=5, time_of_day=11, event_type=u'App_Opened'),
 EventDataRow(userId=u'001e6d8e-cbe7-4374-8c38-f37962a457e9', itemId=u'com.android.phone', ts=1421558502, latitude=47.23505, longitude=-122.534698, city=u'Tacoma', day_of_week=5, time_of_day=21, event_type=u'App_Opened'),
 EventDataRow(userId=u'001e6d8e-cbe7-4374-8c38-f37962a457e9', itemId=u'com.android.systemui', ts=1421017853, latitude=47.237461, longitude=-122.530899, city=u'Tacoma', day_of_week=6, time_of_day=15, event_type=u'App_Opened')]

In [4]:
userIdConversionDictionary = eventsRdd.map(lambda x: x.userId).distinct().zipWithIndex().collectAsMap()
userIdConversionDictionaryBroadcast = sc.broadcast(userIdConversionDictionary)
itemIdConversionDictionary = eventsRdd.map(lambda x: x.itemId).distinct().zipWithIndex().collectAsMap()
itemIdConversionDictionaryBroadcast = sc.broadcast(itemIdConversionDictionary)
cityConversionDictionary = eventsRdd.map(lambda x: x.city).distinct().zipWithIndex().collectAsMap()
cityConversionDictionaryBroadcast = sc.broadcast(cityConversionDictionary)

In [5]:
   eventsConvertedRdd = eventsRdd.map(lambda x: EventDataRow(
    userId=userIdConversionDictionaryBroadcast.value[x.userId],
    itemId=itemIdConversionDictionaryBroadcast.value[x.itemId],
    ts=x.ts,
    latitude=x.latitude,
    longitude=x.longitude,
    city=cityConversionDictionaryBroadcast.value[x.city],
    day_of_week=x.day_of_week,
    time_of_day=x.time_of_day,
    event_type=x.event_type
    ))

eventsConvertedRdd.take(3)   


[EventDataRow(userId=1, itemId=41, ts=1421521691, latitude=47.23505, longitude=-122.534698, city=9, day_of_week=5, time_of_day=11, event_type=u'App_Opened'),
 EventDataRow(userId=1, itemId=46, ts=1421558502, latitude=47.23505, longitude=-122.534698, city=9, day_of_week=5, time_of_day=21, event_type=u'App_Opened'),
 EventDataRow(userId=1, itemId=44, ts=1421017853, latitude=47.237461, longitude=-122.530899, city=9, day_of_week=6, time_of_day=15, event_type=u'App_Opened')]

In [6]:
finalRDD = eventsConvertedRdd.map(lambda x: [
    x.userId,(
    x.itemId,
    x.ts)
    ])
finalRDD.take(3)
#testRDD = allRDD.sample(False, 0.1, 1234)
#testRDD.count()
#trainRDD = allRDD.filter(lambda x: x and x not in testRDD) it is not working
#trainRDD.count()

[[1, (41, 1421521691)], [1, (46, 1421558502)], [1, (44, 1421017853)]]

In [7]:
groupData = map((lambda (x,y): (x, list(y))), sorted(finalRDD.groupByKey().collect()))
#sc.parallelize(groupData).collect()

In [8]:
from random import shuffle

def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def lruFunction(line):
    listGroup = line[1]
    #shuffle(listGroup)                  #shuffle the list
    #listGroup = sorted(listGroup,key=lambda x:int(x[1]), reverse=True); 
    listGroup = sorted(listGroup,key=lambda x:int(x[1])); 
    l = len(listGroup) 
    numTrain = l * 8 / 10
    numTest = l - numTrain
    trainList = listGroup[:numTrain]      #0.8 train set
    testList = listGroup[numTrain:]       #0.2 test set
    
    trainList = sorted(trainList,key=lambda x:int(x[1]), reverse=True);  #sort by timestamp with descending
    testList = [t[0] for t in testList]                      #take only id for test set
    RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
    Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
    
    if len(Recommender) > 4:
        finalRecommender = Recommender[:5]
    else:
        finalRecommender = [-1,-1,-1,-1,-1]
        numRec = len(finalRecommender)
        finalRecommender[:numRec] = Recommender
    
    scores = 0
    numHit = 0
    for t in testList:
        if t == finalRecommender[0]:
            scores = scores+1
            numHit = numHit+1
        elif t == finalRecommender[1]:
            scores = scores+0.8
            numHit = numHit+1
        elif t == finalRecommender[2]:
            scores = scores+0.6
            numHit = numHit+1
        elif t == finalRecommender[3]:
            scores = scores+0.4
            numHit = numHit+1
        elif t == finalRecommender[4]:
            scores = scores+0.2
            numHit = numHit+1

    #scores = scores / numTest
    scores = scores / numHit
    return scores
    #return testList[0],testList[1],testList[2],testList[3],testList[4]
    #if len(Recommender) > 4:
    #    return line[0], Recommender[0], Recommender[1], Recommender[2], Recommender[3], Recommender[4]
    #return line[0],trainList[0],trainList[1],trainList[2],trainList[3],trainList[4],trainList[5],trainList[6],trainList[7]
    #return l,len(trainList),len(testList) 
finalScore = sc.parallelize(groupData).map(lruFunction)
finalScore.collect()

[0.6083333333333331, 0.5428571428571429]

In [9]:
def mruFunction(line):
    listGroup = line[1]
    #shuffle(listGroup)                  #shuffle the list
    #listGroup = sorted(listGroup,key=lambda x:int(x[1]), reverse=True); 
    listGroup = sorted(listGroup,key=lambda x:int(x[1])); 
    l = len(listGroup) 
    numTrain = l * 8 / 10
    numTest = l - numTrain
    trainList = listGroup[:numTrain]      #0.9 train set
    testList = listGroup[numTrain:]       #0.1 test set
    
    #trainList = sorted(trainList,key=lambda x:int(x[1]), reverse=True);  #sort by timestamp with descending
    testList = [t[0] for t in testList]                      #take only id for test set
    RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
    Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
    
    if len(Recommender) > 4:
        finalRecommender = Recommender[:5]
    else:
        finalRecommender = [-1,-1,-1,-1,-1]
        numRec = len(finalRecommender)
        finalRecommender[:numRec] = Recommender
    
    scores = 0
    numHit = 0
    for t in testList:
        if t == finalRecommender[0]:
            scores = scores+1
            numHit = numHit+1
        elif t == finalRecommender[1]:
            scores = scores+0.8
            numHit = numHit+1
        elif t == finalRecommender[2]:
            scores = scores+0.6
            numHit = numHit+1
        elif t == finalRecommender[3]:
            scores = scores+0.4
            numHit = numHit+1
        elif t == finalRecommender[4]:
            scores = scores+0.2
            numHit = numHit+1
        else:
            numHit = numHit+1

    #scores = scores / numTest
    scores = scores / numHit
    return scores
    #return testList[0],testList[1],testList[2],testList[3],testList[4]
    #if len(Recommender) > 4:
    #    return line[0], Recommender[0], Recommender[1], Recommender[2], Recommender[3], Recommender[4]
    #return line[0],trainList[0],trainList[1],trainList[2],trainList[3],trainList[4],trainList[5],trainList[6],trainList[7]
    #return l,len(trainList),len(testList) 
finalScore = sc.parallelize(groupData).map(mruFunction)
finalScore.collect()

[0.2232558139534884, 0.1949367088607594]

In [10]:
from collections import Counter
def mruFunction(line):
    listGroup = line[1]
    #shuffle(listGroup)                  #shuffle the list
    #listGroup = sorted(listGroup,key=lambda x:int(x[1]), reverse=True); 
    listGroup = sorted(listGroup,key=lambda x:int(x[1])); 
    l = len(listGroup) 
    numTrain = l * 8 / 10
    numTest = l - numTrain
    trainList = listGroup[:numTrain]      #0.9 train set
    testList = listGroup[numTrain:]       #0.1 test set
    
    testList = [t[0] for t in testList]                      #take only id for test set
    RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
    #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
    Recommender = Counter(RecommenderDuplicate).most_common()
    Recommender =  [t[0] for t in Recommender] 

    if len(Recommender) > 4:
        finalRecommender = Recommender[:5]
    else:
        finalRecommender = [-1,-1,-1,-1,-1]
        numRec = len(finalRecommender)
        finalRecommender[:numRec] = Recommender
    
    scores = 0
    numHit = 0
    for t in testList:
        if t == finalRecommender[0]:
            scores = scores+1
            numHit = numHit+1
        elif t == finalRecommender[1]:
            scores = scores+0.8
            numHit = numHit+1
        elif t == finalRecommender[2]:
            scores = scores+0.6
            numHit = numHit+1
        elif t == finalRecommender[3]:
            scores = scores+0.4
            numHit = numHit+1
        elif t == finalRecommender[4]:
            scores = scores+0.2
            numHit = numHit+1
        else:
            numHit = numHit+1    

    #scores = scores / numTest
    scores = scores / numHit
    return scores
    #return Recommender
    #return testList[0],testList[1],testList[2],testList[3],testList[4]
    #if len(Recommender) > 4:
    #    return line[0], Recommender[0], Recommender[1], Recommender[2], Recommender[3], Recommender[4]
    #return line[0],trainList[0],trainList[1],trainList[2],trainList[3],trainList[4],trainList[5],trainList[6],trainList[7]
    #return l,len(trainList),len(testList) 
finalScore = sc.parallelize(groupData).map(mruFunction)
finalScore.collect()

[0.4372093023255815, 0.2658227848101266]